# From Delta Lake to Amazon SageMaker

[Delta Lake](https://delta.io/) is a common open-source framework used for storing data in Lakehouse architectures.

In this sample we demonstrate how to integrate Delta Tables with Amazon SageMaker for performing data exploration, ingestion, processing, training, and hosting for Machine Learning.

---

## 2 - Feature Engineering and Ingestion

***Use Kernel "Data Science 3.0 (Python 3)" for running this notebook***

In this notebook, we will ingest data from our Delta Tables, perform some transformations on it via code using **SageMaker Processing**, and ingesting the resulting features into **SageMaker Feature Store**. For this purpose we will:
* Create a SageMaker Feature Store Feature Group, both offline and online
* Prepare a processing script for our feature engineering, including the configuration for connecting to our Delta Table
* Run a SageMaker Processing job pointing towards our sample Delta Table profile file URL. It will include the code for the transformations and ingesting the resulting features into our Feature Group

<center><img src="../images/DeltaLake_to_SageMaker_2.png" width="50%"></center>

Note the transformations to the data can also be performed with other services in AWS, e.g. for low-code/no-code processing you can rely on **SageMaker Data Wrangler**, as it currently supports direct connections towards Delta Lakes via JDBC for data exploration, analysis, and feature engineering. You can check more details about this method in this blog post:

https://aws.amazon.com/blogs/machine-learning/prepare-data-from-databricks-for-machine-learning-using-amazon-sagemaker-data-wrangler/

### Processing data from Delta Lake with SageMaker Processing

In [2]:
import boto3
import sagemaker
from sagemaker import get_execution_role
import pandas as pd
import os
from urllib.parse import urlparse

# S3 bucket for saving processing job outputs
sm_session = sagemaker.Session()
bucket = sm_session.default_bucket()
region = sm_session.boto_region_name

# Delta Sharing profile file location - Replace these with your own if you want to customize this example
table = f's3a://{bucket}/delta_to_sagemaker/delta_format/'
output_path = f's3://{bucket}/delta_to_sagemaker/processing_output/'

role = get_execution_role()

We will start by creating a SageMaker Feature Store Feature Group.

In [160]:
import time
current_time_sec = int(round(time.time()))

features = pd.read_csv('../data/fact_rating_synthetic.csv')
features.drop(['ratingID'], axis=1, inplace=True)
features.columns=['rowID', 'timestamp', 'userID', 'placeID', 'rating_overall', 'rating_food', 'rating_service']
features = features.astype({'rowID': 'int'})
features = features.astype({'userID': 'string'})
features = features.astype({'placeID': 'string'})
features["EventTime"] = pd.Series([current_time_sec] * len(features), dtype="string")

print(features.dtypes)

rowID              int64
timestamp         object
userID            string
placeID           string
rating_overall     int64
rating_food        int64
rating_service     int64
EventTime         string
dtype: object


In [161]:
from sagemaker.feature_store.feature_group import FeatureGroup
feature_group_name = 'rating-fg'

feature_group = FeatureGroup(
    name=feature_group_name,
    sagemaker_session=sm_session,
)

feature_group.load_feature_definitions(features)

[FeatureDefinition(feature_name='rowID', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='timestamp', feature_type=<FeatureTypeEnum.STRING: 'String'>),
 FeatureDefinition(feature_name='userID', feature_type=<FeatureTypeEnum.STRING: 'String'>),
 FeatureDefinition(feature_name='placeID', feature_type=<FeatureTypeEnum.STRING: 'String'>),
 FeatureDefinition(feature_name='rating_overall', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='rating_food', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='rating_service', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='EventTime', feature_type=<FeatureTypeEnum.STRING: 'String'>)]

In [164]:
fg = feature_group.create(
    s3_uri=f's3://{bucket}/delta_to_sagemaker/',
    role_arn=role,
    record_identifier_name='rowID',
    event_time_feature_name='EventTime',
    enable_online_store=True,
)
feature_group_arn = fg['FeatureGroupArn']
print(feature_group_arn)

arn:aws:sagemaker:eu-west-1:889960878219:feature-group/rating-fg


To confirm that your FeatureGroup has been created we use `DescribeFeatureGroup` and `ListFeatureGroups` APIs to display the created FeatureGroup.

In [165]:
feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:eu-west-1:889960878219:feature-group/rating-fg',
 'FeatureGroupName': 'rating-fg',
 'RecordIdentifierFeatureName': 'rowID',
 'EventTimeFeatureName': 'EventTime',
 'FeatureDefinitions': [{'FeatureName': 'rowID', 'FeatureType': 'Integral'},
  {'FeatureName': 'timestamp', 'FeatureType': 'String'},
  {'FeatureName': 'userID', 'FeatureType': 'String'},
  {'FeatureName': 'placeID', 'FeatureType': 'String'},
  {'FeatureName': 'rating_overall', 'FeatureType': 'Integral'},
  {'FeatureName': 'rating_food', 'FeatureType': 'Integral'},
  {'FeatureName': 'rating_service', 'FeatureType': 'Integral'},
  {'FeatureName': 'EventTime', 'FeatureType': 'String'}],
 'CreationTime': datetime.datetime(2023, 2, 8, 12, 2, 17, 52000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': 's3://sagemaker-eu-west-1-889960878219/delta_to_sagemaker/',
   'ResolvedOutputS3Uri': 's3://sagemaker-eu-west-1-88996087

In [166]:
sm_session.boto_session.client(
    "sagemaker", region_name=region
).list_feature_groups(NameContains = 'rating-fg')

{'FeatureGroupSummaries': [{'FeatureGroupName': 'rating-fg',
   'FeatureGroupArn': 'arn:aws:sagemaker:eu-west-1:889960878219:feature-group/rating-fg',
   'CreationTime': datetime.datetime(2023, 2, 8, 12, 2, 17, 52000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Creating'}],
 'NextToken': 'cIws2QhTXUIa8bi8VaXSkZyCip699fykY7CU0a6rxKatwtimPuAdGzYreBOG3pbaFVlWD8bhEvORr3GSVTKiDoNcR3sdCTEgx3jc4+6Ek0uCkSOvfuULbMXz66iJGu/r8djmlM4jg2iBwnGo9Ht5fKZ8B8sPKtRdVLIdvt9uj13vJlrpbXV7iev8qMBw1FSDkS7unEqe2AN+hESbkUGUWcyHG7cbCNr/pEPQ7QDmirfP8/bzGX38hAv2q5+nr4oAT056IYNnvmlLiZJC3yeMW/ZBkCWM8EDh7con5oWO/Vfa3sH81i1K4n4ef8LhBduIhB1XR8yezoxNij7m6T1etOb2zxX96GpzpMzXixyp1uN+QU5XcixEuvpVk5Zo8u4ghZZ7QF9wyiQBA22n1i8QGyEaaRTDdeDaOsKZw16IvwE8PIIR3Gg0FvzIkYhu8km5rjZ3IWSZ7VtLBlWVkizkdX9NFFRNTB5pVtwX/cy4nrxbWLj2oC9R2IKMz7N/xj0sC7iFaoxHagSki4cyE7ACeWM91U5t42eQwux//4NjT/wrtI4O2xTkM3hMMliTRRGgHtuHKyMDl4QcJnbzEaroNk7lHhwS7HgP0A==',
 'ResponseMetadata': {'RequestId': '23646ef0-c5ee-47f1-ae0f-71343e60aa04',
  'HTTPStatusCode':

We can now define our SageMaker Processing job for performing the transformations on our data, and ingesting the results into our Feature Group.

In [167]:
from sagemaker.spark.processing import PySparkProcessor

spark_processor = PySparkProcessor(
    base_job_name="delta-to-sagemaker-",
    framework_version="3.1",
    role=role,
    instance_count=1, #set to >1 for distributed processing
    instance_type="ml.m5.xlarge",
    max_runtime_in_seconds=1200,
)

We will now create our pre-processing script, including code for reading our Delta Table, performing transformations, and ingesting the resulting features into SageMaker Feature Store.

Note in our example we are just including some simple transformations recommended by Data Wrangler on our dataset, but you can replace those with your own transformations if you want to customize this example.

In [168]:
%%writefile ./code/preprocessing.py
import argparse
import csv
import os
import shutil
import sys
import time
import boto3
import pandas as pd
from decimal import Decimal

# Import pyspark and build Spark session
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from feature_store_manager import FeatureStoreManager

def main():
    parser = argparse.ArgumentParser(description="app inputs and outputs")
    parser.add_argument("--region", type=str, help="AWS region")
    parser.add_argument("--table", type=str, help="Delta Table URL")
    parser.add_argument("--feature-group", type=str, help="Name of the Feature Group")
    parser.add_argument("--output-path", type=str, help="S3 prefix for storing resulting dataset")
    args = parser.parse_args()

    # Instantiate Spark via builder
    # Note: we use the `ContainerCredentialsProvider` to give us access to underlying IAM role permissions
    spark = (SparkSession
        .builder
        .appName("PySparkApp") 
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
        .config("fs.s3a.aws.credentials.provider",'com.amazonaws.auth.ContainerCredentialsProvider') 
        .getOrCreate())

    sc = spark.sparkContext
    print('Spark version: '+str(sc.version))
    
    s3a_delta_table_uri=args.table
    print(s3a_delta_table_uri)

    # Create SQL command inserting the S3 path location
    sql_cmd = f'SELECT * FROM delta.`{s3a_delta_table_uri}` ORDER BY timestamp'
    print(f'SQL command: {sql_cmd}')

    # Execute SQL command which returns dataframe
    sql_results = spark.sql(sql_cmd)
    print(type(sql_results))

    # ----------------
    # Transformations - Pandas code generated by sagemaker_datawrangler:
    processed_features = sql_results.toPandas().copy(deep=True)

    # Code to Replace with new value for column: userID to resolve warning: Disguised missing values 
    generic_value = 'Other'
    processed_features['userID']=processed_features['userID'].replace('na', 'Other', regex=False)
    processed_features['userID']=processed_features['userID'].replace('nA', 'Other', regex=False)

    # Code to Drop column for column: ratingID to resolve warning: ID column 
    processed_features=processed_features.drop(columns=['ratingID'])
    
    # Complete with EventTime feature
    processed_features['EventTime']=str(pd.to_datetime('now').strftime('%Y-%m-%dT%H:%M:%SZ'))

    print(processed_features.head())

    # ----------------
    # Capture resulting data frame in Spark:
    columns = ['rowID', 'timestamp', 'userID', 'placeID', 'rating_overall', 'rating_food', 'rating_service', 'EventTime']
    df = spark.createDataFrame(processed_features).toDF(*columns)
    
    # Write processed data after transformations to S3...
    processed_features_output_path = args.output_path + 'processed_features.csv'
    print("Saving processed features to {}".format(processed_features_output_path))
    df.write.csv(processed_features_output_path)

    # Ingesting the resulting data into our Feature Group...
    print(f"Ingesting processed features into Feature Group {args.feature_group}...")
    feature_store_manager = FeatureStoreManager()
    feature_store_manager.ingest_data(
        input_data_frame=df,
        feature_group_arn=f'{args.feature_group}',
        target_stores=['OfflineStore']
    )

    print("All done.")

if __name__ == "__main__":
    main()

Overwriting ./code/preprocessing.py


In our example we are using the SageMaker Spark container as a base, so we will just include the additional "delta-core" library as an additional JAR file, together with the "feature_store_manager" required for ingesting into the Feature Group at scale.

In [169]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

spark_processor.run(
    submit_app='./code/preprocessing.py',
    submit_jars=[
        './delta-core_2.12-1.0.1.jar',
        './feature_store_pyspark/sagemaker-feature-store-spark-sdk.jar'
    ],
    submit_py_files=[
        './feature_store_pyspark/feature_store_manager.py',
        './feature_store_pyspark/wrapper.py'
    ],
    arguments=[
        '--region', region,
        '--table', table,
        '--feature-group', feature_group_arn,
        '--output-path', output_path, 
    ],
)

INFO:sagemaker.spark.processing:Copying dependency from local path ./feature_store_pyspark/feature_store_manager.py to tmpdir /tmp/tmp6yz2oms6
INFO:sagemaker.spark.processing:Copying dependency from local path ./feature_store_pyspark/wrapper.py to tmpdir /tmp/tmp6yz2oms6
INFO:sagemaker.spark.processing:Uploading dependencies from tmpdir /tmp/tmp6yz2oms6 to S3 s3://sagemaker-eu-west-1-889960878219/delta-to-sagemaker--2023-02-08-12-02-28-379/input/py-files
INFO:sagemaker.spark.processing:Copying dependency from local path ./delta-core_2.12-1.0.1.jar to tmpdir /tmp/tmpf9old2ip
INFO:sagemaker.spark.processing:Copying dependency from local path ./feature_store_pyspark/sagemaker-feature-store-spark-sdk.jar to tmpdir /tmp/tmpf9old2ip
INFO:sagemaker.spark.processing:Uploading dependencies from tmpdir /tmp/tmpf9old2ip to S3 s3://sagemaker-eu-west-1-889960878219/delta-to-sagemaker--2023-02-08-12-02-28-379/input/jars
INFO:sagemaker:Creating processing-job with name delta-to-sagemaker--2023-02-08-

.........................02-08 12:06 smspark.cli  INFO     Parsing arguments. argv: ['/usr/local/bin/smspark-submit', '--py-files', '/opt/ml/processing/input/py-files', '--jars', '/opt/ml/processing/input/jars', '/opt/ml/processing/input/code/preprocessing.py', '--region', 'eu-west-1', '--table', 's3a://sagemaker-eu-west-1-889960878219/delta_to_sagemaker/delta_format/', '--feature-group', 'arn:aws:sagemaker:eu-west-1:889960878219:feature-group/rating-fg', '--output-path', 's3://sagemaker-eu-west-1-889960878219/delta_to_sagemaker/processing_output/']
02-08 12:06 smspark.cli  INFO     Raw spark options before processing: {'py_files': '/opt/ml/processing/input/py-files', 'jars': '/opt/ml/processing/input/jars', 'class_': None, 'files': None, 'verbose': False}
02-08 12:06 smspark.cli  INFO     App and app arguments: ['/opt/ml/processing/input/code/preprocessing.py', '--region', 'eu-west-1', '--table', 's3a://sagemaker-eu-west-1-889960878219/delta_to_sagemaker/delta_format/', '--feature-gro

Let's get one of the processed files generated.

In [175]:
boto_session = boto3.Session()
s3 = boto_session.resource('s3')
my_bucket = s3.Bucket(bucket)

columns=['rowID', 'timestamp', 'userID', 'placeID', 'rating_overall', 'rating_food', 'rating_service', 'EventTime']
for objects in my_bucket.objects.filter(Prefix="delta_to_sagemaker/processing_output/processed_features.csv/"):
    if objects.key.endswith('csv'):
        print(objects.key)
        processed_features = pd.read_csv(f's3://{bucket}/{objects.key}', index_col=None, header=None, names=columns)
        break
processed_features.head()


delta_to_sagemaker/processing_output/processed_features.csv/part-00000-1afdfd31-2157-46b9-a8b7-576e7744659b-c000.csv


,rowID,timestamp,userID,placeID,rating_overall,rating_food,rating_service,EventTime
0,0,2022-08-25,gK,681,1,2,2,2023-02-08T12:07:34Z
1,1,2022-08-25,gK,719,1,1,1,2023-02-08T12:07:34Z
2,2,2022-08-25,gK,1128,1,2,2,2023-02-08T12:07:34Z
3,3,2022-08-25,gK,1203,1,2,2,2023-02-08T12:07:34Z
4,4,2022-08-25,gK,1058,1,1,1,2023-02-08T12:07:34Z


----

### Verifying processed data in SageMaker Feature Store

Using an arbirary customer record id, 57 we use `get_record` to check that the data has been ingested into the feature group.

In [171]:
sample_record = sm_session.boto_session.client(
    "sagemaker-featurestore-runtime", region_name=region
).get_record(
    FeatureGroupName=feature_group_name,
    RecordIdentifierValueAsString='1'
)

In [172]:
sample_record

{'ResponseMetadata': {'RequestId': 'faf10574-9f48-4f51-9b7f-8166503bbd75',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'faf10574-9f48-4f51-9b7f-8166503bbd75',
   'content-type': 'application/json',
   'content-length': '15',
   'date': 'Wed, 08 Feb 2023 12:08:36 GMT'},
  'RetryAttempts': 0}}

We can also use `batch_get_record` to check multiple records ingested into the feature groups by providing customer ids.

In [173]:
all_records = sm_session.boto_session.client(
    'sagemaker-featurestore-runtime', region_name=region
).batch_get_record(
    Identifiers=[
        {
            'FeatureGroupName': feature_group_name,
            'RecordIdentifiersValueAsString': ['5', '3', '8'],
        }
    ]
)

In [174]:
all_records

{'ResponseMetadata': {'RequestId': '528a11a7-1e32-4434-97d0-af3dd175f9ab',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '528a11a7-1e32-4434-97d0-af3dd175f9ab',
   'content-type': 'application/json',
   'content-length': '54',
   'date': 'Wed, 08 Feb 2023 12:08:40 GMT'},
  'RetryAttempts': 0},
 'Records': [],
 'Errors': [],
 'UnprocessedIdentifiers': []}

----------

### (Optional) Clean-up

In [162]:
#Delete the feature group
!aws sagemaker delete-feature-group --feature-group-name rating-fg

In [163]:
#Delete the copy of the processed data in S3
!aws s3 rm --recursive s3://{bucket}/delta_to_sagemaker/processing_output/processed_features.csv/

delete: s3://sagemaker-eu-west-1-889960878219/delta_to_sagemaker/processing_output/processed_features.csv/part-00000-3638c6ad-7686-46f1-98d6-d9ea30e487cd-c000.csv
delete: s3://sagemaker-eu-west-1-889960878219/delta_to_sagemaker/processing_output/processed_features.csv/_SUCCESS
delete: s3://sagemaker-eu-west-1-889960878219/delta_to_sagemaker/processing_output/processed_features.csv/part-00002-3638c6ad-7686-46f1-98d6-d9ea30e487cd-c000.csv
delete: s3://sagemaker-eu-west-1-889960878219/delta_to_sagemaker/processing_output/processed_features.csv/part-00001-3638c6ad-7686-46f1-98d6-d9ea30e487cd-c000.csv
delete: s3://sagemaker-eu-west-1-889960878219/delta_to_sagemaker/processing_output/processed_features.csv/part-00005-3638c6ad-7686-46f1-98d6-d9ea30e487cd-c000.csv
delete: s3://sagemaker-eu-west-1-889960878219/delta_to_sagemaker/processing_output/processed_features.csv/part-00004-3638c6ad-7686-46f1-98d6-d9ea30e487cd-c000.csv
delete: s3://sagemaker-eu-west-1-889960878219/delta_to_sagemaker/proce